In [3]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Source #Partner

### Links: 
___

In [ ]:
# creating a table from s3

query_text = """--sql
    CREATE TABLE db1.entries_installation_points_dir_partner
    (
        `full_address` String,
        `created_at` String,
        `number` Int32,
        `lat` String,
        `lon` String,
        `first_flat` Int16,
        `last_flat` Int16,
        `flats_count` Int16,
        `address_uuid` String,
        `parent_uuid` String,
        `partner_uuid` String,
        `installation_point_id` Int64,
        `region` String,
        `country` String,
        `city` String,
        `city_uuid` String
    )
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/entries_installation_points_dir_partner/entries_installation_points_dir_partner.csv','CSVWithNames')
    """

ch.query_run(query_text)

In [10]:
query_text = """--sql
   CREATE TABLE db1.entries_installation_points_dir_partner_ch
    (
        `full_address` String,
        `created_at` String,
        `number` Int32,
        `lat` String,
        `lon` String,
        `first_flat` Int16,
        `last_flat` Int16,
        `flats_count_full` Int16,
        `flats_count` Int16,
        `address_uuid` String,
        `parent_uuid` String,
        `partner_uuid` String,
        `installation_point_id` Int64,
        `region` String,
        `country` String,
        `city` String,
        `city_uuid` String,
        `parts` Array(String),
        `entrance_number` String,
        `building_number` String,
        `streat_name` String        
    )
    ENGINE = MergeTree()
    ORDER BY installation_point_id
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

Количество квартир в базе отличается от квартир на подъездах

In [11]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.entries_installation_points_dir_partner_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.entries_installation_points_dir_partner_ch AS
    SELECT
	    `full_address`,
	    `created_at`,
	    `number`,
	    `lat`,
	    `lon`,
	    `first_flat`,
	    `last_flat`,
	    `flats_count`,
	    `last_flat` - `first_flat` + 1 AS `flats_count_full`,
	    `address_uuid`,
	    `parent_uuid`,
	    `partner_uuid`,
	    `installation_point_id`,
	    `region`,
	    `country`,
	    `city`,
		`city_uuid`,
		splitByString(', ', full_address) AS parts,
        parts[-1] AS entrance_number,
        parts[-2] AS building_number,
        parts[-3] AS streat_name 
	FROM db1.entries_installation_points_dir_partner
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query

In [12]:
query_text = """--sql
    SELECT
        *
    FROM db1.entries_installation_points_dir_partner_ch
    limit 10
    """

ch.query_run(query_text)

full_address,created_at,number,lat,lon,first_flat,last_flat,flats_count_full,flats_count,address_uuid,parent_uuid,partner_uuid,installation_point_id,region,country,city,city_uuid,parts,entrance_number,building_number,streat_name
str,str,i64,str,str,i64,i64,i64,i64,str,str,str,i64,str,str,str,str,list[str],str,str,str
"""Россия, Республика Татарстан, …","""2018-05-25 07:50:25""",1,"""55.747789""","""48.740027""",1,26,26,20,"""383d3359-0c4b-47a2-ab65-a80faf…","""54086eea-e2b4-46ad-ba06-913233…","""""",1,"""Республика Татарстан""","""Россия""","""Иннополис""","""bcaf46fc-a7f2-4c68-8692-cd1946…","[""Россия"", ""Республика Татарстан"", … ""п. 1""]","""п. 1""","""д. 100""","""ул. Спортивная"""
"""Россия, Республика Татарстан, …","""2018-05-25 07:50:25""",2,"""55.747839""","""48.740054""",1,55,55,26,"""a6b2cde0-26b5-4ce3-af2e-8d03e3…","""54086eea-e2b4-46ad-ba06-913233…","""""",2,"""Республика Татарстан""","""Россия""","""Иннополис""","""bcaf46fc-a7f2-4c68-8692-cd1946…","[""Россия"", ""Республика Татарстан"", … ""п. 2""]","""п. 2""","""д. 100""","""ул. Спортивная"""
"""Россия, Республика Татарстан, …","""2018-05-25 07:50:25""",3,"""55.747941""","""48.741698""",56,85,30,24,"""19423336-12a2-47c0-ab44-7dbc53…","""54086eea-e2b4-46ad-ba06-913233…","""""",3,"""Республика Татарстан""","""Россия""","""Иннополис""","""bcaf46fc-a7f2-4c68-8692-cd1946…","[""Россия"", ""Республика Татарстан"", … ""п. 3""]","""п. 3""","""д. 100""","""ул. Спортивная"""
"""Россия, Республика Татарстан, …","""2018-05-25 07:50:25""",1,"""55.747069""","""48.743054""",1,25,25,22,"""6cfebc6c-0daa-4224-97f6-82c365…","""19387fa7-04d8-4637-98a3-1ee2de…","""""",4,"""Республика Татарстан""","""Россия""","""Иннополис""","""bcaf46fc-a7f2-4c68-8692-cd1946…","[""Россия"", ""Республика Татарстан"", … ""п. 1""]","""п. 1""","""д. 104""","""ул. Спортивная"""
"""Россия, Республика Татарстан, …","""2018-05-25 07:50:25""",2,"""55.747201""","""48.743171""",1,55,55,24,"""29a1ab4c-cf49-4942-8732-a617f0…","""19387fa7-04d8-4637-98a3-1ee2de…","""""",5,"""Республика Татарстан""","""Россия""","""Иннополис""","""bcaf46fc-a7f2-4c68-8692-cd1946…","[""Россия"", ""Республика Татарстан"", … ""п. 2""]","""п. 2""","""д. 104""","""ул. Спортивная"""
"""Россия, Республика Татарстан, …","""2018-05-25 07:50:25""",3,"""55.747434""","""48.743396""",51,85,35,28,"""c31a364c-cc21-483d-9dd4-754e74…","""19387fa7-04d8-4637-98a3-1ee2de…","""""",6,"""Республика Татарстан""","""Россия""","""Иннополис""","""bcaf46fc-a7f2-4c68-8692-cd1946…","[""Россия"", ""Республика Татарстан"", … ""п. 3""]","""п. 3""","""д. 104""","""ул. Спортивная"""
"""Россия, Республика Татарстан, …","""2018-05-25 07:50:25""",1,"""55.748017""","""48.742659""",1,25,25,23,"""501b7f66-eb3d-410f-96d0-da75fc…","""172aff05-9d66-4f98-b485-1f177d…","""""",7,"""Республика Татарстан""","""Россия""","""Иннополис""","""bcaf46fc-a7f2-4c68-8692-cd1946…","[""Россия"", ""Республика Татарстан"", … ""п. 1""]","""п. 1""","""д. 110""","""ул. Спортивная"""
"""Россия, Республика Татарстан, …","""2018-05-25 07:50:25""",2,"""55.74786""","""48.742515""",26,55,30,28,"""82ae8710-c673-4ddd-b8b0-d88eb3…","""172aff05-9d66-4f98-b485-1f177d…","""""",8,"""Республика Татарстан""","""Россия""","""Иннополис""","""bcaf46fc-a7f2-4c68-8692-cd1946…","[""Россия"", ""Республика Татарстан"", … ""п. 2""]","""п. 2""","""д. 110""","""ул. Спортивная"""
"""Россия, Республика Татарстан, …","""2018-05-25 07:50:25""",3,"""55.747637""","""48.742318""",56,85,30,21,"""029d2d1d-792e-4b43-8245-6c4693…","""172aff05-9d66-4f98-b485-1f177d…","""""",9,"""Республика Татарстан""","""Россия""","""Иннополис""","""bcaf46fc-a7f2-4c68-8692-cd1946…","[""Россия"", ""Республика Татарстан"", … ""п. 3""]","""п. 3""","""д. 110""","""ул. Спортивная"""


### Drop ch

In [8]:
query_text = """
    DROP TABLE db1.entries_installation_points_dir_partner_ch
    """
ch.query_run(query_text)

elapsed_ns,query_id
i64,str
5869116,"""3398d001-5662-4cb4-bfcb-cc56ac…"


### Drop mv

In [9]:
query_text = """
    DROP TABLE db1.entries_installation_points_dir_partner_mv
    """
ch.query_run(query_text)

elapsed_ns,query_id
i64,str
5062862,"""0dae5209-df96-439c-b3d6-254d25…"
